In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movierating/movieRating.csv


In [2]:
# 匯入必要的套件
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dot
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error

In [3]:
# 讀取資料
file_path = "/kaggle/input/movierating/movieRating.csv"
movie_data = pd.read_csv(file_path)


In [4]:
# 隨機打散並分割資料
train_data, test_data = train_test_split(movie_data, test_size=0.2, random_state=42, shuffle=True)

# 映射 UserID 和 MovieID 到索引
user_ids = movie_data['UserID'].unique()
movie_ids = movie_data['MovieID'].unique()

user_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
movie_to_index = {movie_id: idx for idx, movie_id in enumerate(movie_ids)}

In [5]:
# 新增索引欄位
train_data['UserIndex'] = train_data['UserID'].map(user_to_index)
train_data['MovieIndex'] = train_data['MovieID'].map(movie_to_index)
test_data['UserIndex'] = test_data['UserID'].map(user_to_index)
test_data['MovieIndex'] = test_data['MovieID'].map(movie_to_index)

In [6]:
# 定義矩陣分解模型
n_users = len(user_ids)
n_movies = len(movie_ids)
embedding_size = 50

user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))

# 移除 `input_length` 參數
user_embedding = Embedding(input_dim=n_users, output_dim=embedding_size)(user_input)
movie_embedding = Embedding(input_dim=n_movies, output_dim=embedding_size)(movie_input)

user_vec = Flatten()(user_embedding)
movie_vec = Flatten()(movie_embedding)

dot_product = Dot(axes=1)([user_vec, movie_vec])

model = Model(inputs=[user_input, movie_input], outputs=dot_product)
model.compile(optimizer=Adam(), loss='mean_absolute_error')

In [7]:
# 準備訓練與測試資料
X_train = [train_data['UserIndex'].values, train_data['MovieIndex'].values]
y_train = train_data['Rating'].values

X_test = [test_data['UserIndex'].values, test_data['MovieIndex'].values]
y_test = test_data['Rating'].values

In [8]:
# 訓練模型
model.fit(X_train, y_train, epochs=5, batch_size=256, verbose=1, validation_split=0.1)

# 預測與計算 MAE
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")

Epoch 1/5


I0000 00:00:1733230955.000334      67 service.cc:145] XLA service 0x7f38d0004750 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733230955.000390      67 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1733230955.000394      67 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 133/2531 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 3.5801

I0000 00:00:1733230955.633699      67 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2531/2531 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 2.6960 - val_loss: 0.7713
Epoch 2/5
2531/2531 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.7469 - val_loss: 0.7318
Epoch 3/5
2531/2531 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.7167 - val_loss: 0.7213
Epoch 4/5
2531/2531 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.6968 - val_loss: 0.7112
Epoch 5/5
2531/2531 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.6768 - val_loss: 0.7048
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step
Mean Absolute Error (MAE): 0.7039015975243027
